In [14]:
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
import multiprocessing
from data_loader import data_loader
from tqdm import tqdm
from functools import partial

In [19]:
def data_loader_all(func, path, train, nrows, **kwargs):
    '''
    Parameters:
    
    func: 하나의 csv파일을 읽는 함수 
    path: [str] train용 또는 test용 csv 파일들이 저장되어 있는 폴더 
    train: [boolean] train용 파일들 불러올 시 True, 아니면 False
    nrows: [int] csv 파일에서 불러올 상위 n개의 row 
    lookup_table: [pd.DataFrame] train_label.csv 파일을 저장한 변수 
    event_time: [int] 상태_B 발생 시간 
    normal: [int] 상태_A의 라벨
    
    Return:
    
    combined_df: 병합된 train 또는 test data
    '''
    files_in_dir = os.listdir(path)
    
    files_path = [path+'/'+file for file in files_in_dir]
    
    if train :
        func_fixed = partial(func, nrows = nrows, train = True, lookup_table = kwargs['lookup_table'], event_time = kwargs['event_time'], normal = kwargs['normal'])
        
    else : 
        func_fixed = partial(func, nrows = nrows, train = False)
    
    
    # 여러개의 코어를 활용하여 데이터 읽기 
    if __name__ == '__main__':
        pool = Pool(processes = multiprocessing.cpu_count()) 
        df_list = list(tqdm(pool.imap(func_fixed, files_path), total = len(files_path)))
        pool.close()
        pool.join()
    
    # 데이터 병합하기 
    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df
    
    

In [20]:
train_path = "./data/train"
test_path = "./data/test"
label = pd.read_csv("./data/train_label.csv")
train = data_loader_all(data_loader, path=train_path, train=True, 
                       nrows=100, normal=999, event_time=10, lookup_table=label)


 80%|████████  | 665/828 [00:55<00:16, 10.10it/s]


100%|██████████| 828/828 [01:08<00:00, 12.16it/s]


In [21]:
test = data_loader_all(data_loader, path=test_path, train=False, nrows=60)


 92%|█████████▏| 664/720 [00:55<00:08,  6.56it/s]


100%|██████████| 720/720 [01:00<00:00, 11.88it/s]


In [24]:
train.head(20)

,id,time,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,V0009,V0010,V0011,V0012,V0013,V0014,V0015,V0016,V0017,V0018,V0019,V0020,V0021,V0022,V0023,V0024,V0025,V0026,V0027,V0028,V0029,V0030,V0031,V0032,V0033,V0034,V0035,V0036,V0037,...,V5082,V5083,V5084,V5085,V5086,V5087,V5088,V5089,V5090,V5091,V5092,V5093,V5094,V5095,V5096,V5097,V5098,V5099,V5100,V5101,V5102,V5103,V5104,V5105,V5106,V5107,V5108,V5109,V5110,V5111,V5112,V5113,V5114,V5115,V5116,V5117,V5118,V5119,V5120,label
0,545,0,30.465864,8.769276,8.718136,8.649309,8.666047,209.350624,183.481557,-1.750378e-19,0.0,0.000553,0.0,5.088343e-19,-1.488497e-18,274.383230,274.515063,274.384768,274.481366,36.020397,36.025485,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.331118,2.574077,2.567163,35.987404,36.003400,36.004403,36.011688,36.003243,36.009225,-0.0001,-0.0001,-0.0001,-0.0001,...,1,1,110.934948,110.927010,-0.299664,-0.225182,-0.217564,-0.133897,43.197349,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,60,127.58,112.263,-0.264680,85.4,-0.004474,999
1,545,1,30.469066,8.883353,8.728539,8.763734,8.716318,182.685628,192.419782,1.935626e-19,0.0,0.000704,0.0,-4.200054e-19,-1.175089e-19,273.804343,273.599199,274.725471,273.316115,36.004735,36.149523,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.336579,2.572470,2.566424,35.989795,35.970029,36.030450,36.009722,36.009290,35.993855,-0.0001,-0.0001,-0.0001,-0.0001,...,1,1,110.812783,110.844505,-0.294121,-0.244735,-0.192494,-0.143298,43.194442,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,60,0.00,0.000,0.000024,85.4,0.000000,999
2,545,2,30.471939,8.750871,8.733991,8.694832,8.692292,204.660928,166.195751,-2.207702e-19,0.0,0.001845,0.0,-1.599024e-18,3.438004e-18,274.589877,273.740876,275.536315,274.991482,35.988678,36.052460,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.343059,2.573650,2.567116,35.983719,35.977091,36.008901,35.998901,36.019894,35.997466,-0.0001,-0.0001,-0.0001,-0.0001,...,1,1,110.893933,110.880650,-0.303727,-0.224236,-0.187819,-0.157744,43.197233,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,60,0.00,0.000,-0.000011,85.4,0.000000,999
3,545,3,30.457942,8.797687,8.732132,8.729372,8.701479,179.748847,206.193738,-4.594139e-19,0.0,0.000008,0.0,-2.733400e-18,-3.330696e-18,274.587881,273.269740,275.098710,274.585110,35.973358,35.920162,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.350158,2.572818,2.566492,35.998485,35.990846,36.003820,35.970836,35.987359,36.014393,-0.0001,-0.0001,-0.0001,-0.0001,...,1,1,110.807798,110.792822,-0.318033,-0.233167,-0.240736,-0.169004,43.204989,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,60,0.00,0.000,-0.000002,85.4,0.000000,999
4,545,4,30.453427,8.686378,8.712109,8.704956,8.687103,190.937355,163.864319,1.856829e-19,0.0,-0.000190,0.0,-8.005414e-19,-3.717984e-18,272.595793,273.294081,273.197602,273.826731,36.006156,35.953997,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.345441,2.573922,2.565957,36.005578,35.978993,36.014680,35.981198,36.003260,35.995221,-0.0001,-0.0001,-0.0001,-0.0001,...,1,1,110.904139,110.703885,-0.284998,-0.258391,-0.230091,-0.176348,43.199886,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,60,0.00,0.000,-0.000031,85.4,0.000000,999
5,545,5,30.474984,8.825343,8.709057,8.712641,8.715015,189.345794,190.109292,3.132490e-19,0.0,0.000849,0.0,-8.002305e-19,1.994740e-18,272.988179,274.117711,275.188731,275.402225,35.971505,36.046793,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.343633,2.572462,2.566821,36.005120,35.994730,35.992584,36.004472,35.998402,36.027051,-0.0001,-0.0001,-0.0001,-0.0001,...,1,1,110.871683,110.968968,-0.284302,-0.257030,-0.174257,-0.150411,43.204300,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,60,0.00,0.000,-0.000003,85.4,0.000000,999
6,545,6,30.470703,8.777491,8.690715,8.712619,8.712374,184.385638,222.773946,5.214632e-19,0.0,-0.000689,0.0,-7.956155e-19,-7.741531e-18,272.998107,273.548580,273.395569,273.619796,36.013859,35.962767,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.354015,2.573795,2.566675,35.991876,36.020052,35.

In [23]:
train.shape

(82800, 5124)

In [26]:
train.describe()

,id,time,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,V0009,V0010,V0011,V0012,V0013,V0014,V0015,V0016,V0017,V0018,V0025,V0026,V0027,V0028,V0029,V0030,V0031,V0032,V0033,V0040,V0041,V0042,V0043,V0044,V0045,V0046,V0047,V0048,V0049,...,V5005,V5006,V5007,V5008,V5009,V5010,V5011,V5012,V5013,V5014,V5015,V5016,V5017,V5018,V5019,V5020,V5021,V5022,V5023,V5024,V5025,V5074,V5075,V5076,V5077,V5078,V5079,V5080,V5084,V5085,V5086,V5087,V5088,V5089,V5090,V5116,V5117,V5118,V5120,label
count,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,8.280000e+04,82800.000000,82800.000000,82800.000000,8.280000e+04,8.280000e+04,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,...,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,82800.000000,8.280000e+04,82800.000000,82800.000000
mean,413.500000,49.500000,30.304857,8.685028,8.659633,8.650845,8.653157,190.272951,189.734731,1.394066e+01,8.807388,14.458604,9.382300,1.445853e+01,9.382817e+00,268.970524,268.547715,275.538183,275.078386,36.204637,36.219846,18.342865,2.572937,2.565418,36.215543,36.215540,36.217333,36.215469,36.215515,36.217194,18.005366,2.519558,2.274609,18.111676,2.620700,2.275872,14.785188,18.006780,2.231807,2.239265,...,86.090330,85.958932,86.401494,86.401554,86.401462,86.401945,119.917910,119.914508,119.904589,119.906141,179.868002,179.867010,179.835511,43.200017,43.200024,43.200010,43.200010,43.200019,43.200009,43.200028,43.200014,578.556208,42.210547,385.471056,27.880552,550.804276,550.804591,43.200014,110.882804,110.882541,-0.293302,-0.235954,-0.195620,-0.153251,43.199965,1.391700,1.234489,1.363329e+00,-0.000017,188.152174
std,239.024281,28.866244,1.411179,0.935292,0.929270,0.973276,0.973147,47.248264,52.177493,5.889067e+01,47.130024,59.815806,48.630435,5.981582e+01,4.863034e+01,37.765112,37.717512,9.121809,9.198233,4.147750,4.133652,0.113520,0.009881,0.019019,4.133519,4.133492,4.133601,4.133506,4.133530,4.133548,0.052136,0.024299,0.017097,0.039196,0.002926,0.019340,0.042734,0.014093,0.005736,0.002060,...,5.174266,6.167349,0.046160,0.052245,0.047118,0.056985,2.373932,2.461957,2.750886,2.705085,3.793286,3.831951,4.734604,0.000488,0.002735,0.002755,0.000274,0.002761,0.002723,0.002766,0.000393,116.880784,13.401253,78.134158,8.953295,0.382158,0.389594,0.005749,0.110005,0.106987,0.048773,0.008456,0.042124,0.118797,0.005599,13.225114,11.731026,9.729998e+00,0.000889,274.991128
min,0.000000,0.000000,0.000000,4.027708,4.189661,3.693100,3.700223,-52.879274,-88.369326,-1.318431e-01,0.000000,-0.191165,0.000000,-2.570846e-01,-2.827205e-17,-2.973148,-4.770904,265.355180,264.494881,20.514291,20.636324,15.065489,2.288562,2.017749,20.503711,20.528395,20.557344,20.537542,20.530780,20.544539,16.508758,2.041370,1.779895,16.988256,2.537952,2.271050,14.783688,17.679629,2.067188,2.236006,...,-0.059782,-0.069080,86.310605,86.289952,86.291243,86.309241,51.637500,49.038754,40.736254,42.086246,70.653060,69.609370,43.571786,43.200000,43.188082,43.188584,43.200000,43.187940,43.189282,43.186175,43.200000,-50.468388,-33.207797,-33.158714,-15.668496,549.276122,549.211010,43.175900,110.571970,110.618648,-0.333704,-0.271033,-0.316519,-0.221357,43.176846,0.000000,0.000000,-2.647478e-01,-0.004474,0.000000
25%,206.750000,24.750000,30.463846,8.666677,8.690121,8.680818,8.6